In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tiwiter/twitter_toxic_tweets.csv


In [2]:
# Toxic Tweets Analysis using Pandas
A complete end-to-end data analysis project built for Kaggle using Pandas.


SyntaxError: invalid syntax (1800417899.py, line 2)

In [ ]:
## Objective
- Load and explore the dataset
- Clean and preprocess text data
- Perform exploratory data analysis
- Generate insights using Pandas


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


In [ ]:
df = pd.read_csv("/kaggle/input/tiwiter/twitter_toxic_tweets.csv")


In [ ]:
df.shape


In [ ]:
df.columns


In [ ]:
df.head()


In [ ]:
df.tail()


In [ ]:
df.info()


In [ ]:
df.dtypes


In [ ]:
df.isnull().sum()


In [ ]:
(df.isnull().mean() * 100)


In [ ]:
df = df.drop_duplicates()


In [ ]:
df.shape


In [ ]:
df.rename(columns=lambda x: x.lower(), inplace=True)


In [ ]:
df['label'].unique()


In [ ]:
df['label'].value_counts()


In [ ]:
df['label'].value_counts(normalize=True) * 100


In [ ]:
df['label'].value_counts().plot(kind='bar', title='Label Distribution')
plt.show()


In [ ]:
df['text_length'] = df['tweet'].astype(str).apply(len)


In [ ]:
df[['tweet', 'text_length']].head()


In [ ]:
df['text_length'].describe()


In [ ]:
df['text_length'].hist(bins=50)
plt.title("Tweet Length Distribution")
plt.show()


In [ ]:
df.groupby('label')['text_length'].mean()


In [ ]:
df.loc[df['text_length'].idxmax()]


In [ ]:
df.loc[df['text_length'].idxmin()]


In [ ]:
df['word_count'] = df['tweet'].apply(lambda x: len(x.split()))


In [ ]:
df['word_count'].describe()


In [ ]:
df.groupby('label')['word_count'].mean()


In [ ]:
df['word_count'].hist(bins=40)
plt.show()


In [ ]:
df['tweet_clean'] = df['tweet'].str.lower()


In [ ]:
df['tweet_clean'] = df['tweet_clean'].str.replace(r'http\S+', '', regex=True)


In [ ]:
df['tweet_clean'] = df['tweet_clean'].str.replace(r'[^\w\s]', '', regex=True)


In [ ]:
df['tweet_clean'] = df['tweet_clean'].str.replace(r'\d+', '', regex=True)


In [ ]:
df['tweet_clean'] = df['tweet_clean'].str.strip()


In [ ]:
df[['tweet', 'tweet_clean']].head()


In [ ]:
toxic_df = df[df['label'] == 1]


In [ ]:
non_toxic_df = df[df['label'] == 0]


In [ ]:
len(toxic_df), len(non_toxic_df)


In [ ]:
df.groupby('label')['text_length'].mean()


In [ ]:
df.groupby('label')['word_count'].mean()


In [ ]:
toxic_words = " ".join(toxic_df['tweet_clean']).split()
pd.Series(toxic_words).value_counts().head(20)


In [ ]:
non_toxic_words = " ".join(non_toxic_df['tweet_clean']).split()
pd.Series(non_toxic_words).value_counts().head(20)


In [ ]:
df.to_csv('clean_toxic_tweets.csv', index=False)


In [ ]:
df.columns


In [ ]:
df['rolling_mean'] = df['text_length'].rolling(window=7).mean()
df['rolling_std']  = df['text_length'].rolling(window=5).std()
df['rolling_sum']  = df['text_length'].rolling(window=10).sum()


In [ ]:
# List all columns clearly
for col in df.columns:
    print(col)


In [ ]:
df.select_dtypes(include='number').columns.tolist()


In [ ]:
# Pick the first numeric column automatically
num_col = df.select_dtypes(include='number').columns[0]

# Ensure numeric
df[num_col] = pd.to_numeric(df[num_col], errors='coerce')

# Rolling mean
df[f'rolling_{num_col}_mean'] = (
    df[num_col]
    .rolling(window=7, min_periods=1)
    .mean()
)


In [ ]:
df[[num_col, f'rolling_{num_col}_mean']].head(10)


In [ ]:
df['rolling_text_length_mean'] = (
    df['text_length']
    .rolling(window=7, min_periods=1)
    .mean()
)


In [ ]:
df.columns.tolist()


In [ ]:
# View all column names
df.columns.tolist()


In [ ]:
[c for c in df.columns if 'like' in c.lower()]


In [ ]:
pd.pivot_table(df, values='text_length', index='label', aggfunc='mean')


## Key Insights
- Toxic tweets are longer on average
- Toxic tweets contain more aggressive vocabulary
- Dataset is imbalanced


## Conclusion
This project demonstrates complete Pandas-based data analysis suitable for Kaggle.


- Apply NLP models
- Feature engineering
- Build ML classifier


Skills Covered:
- Pandas
- Data Cleaning
- EDA
- Visualization


# End of Project


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [ ]:
X = df['tweet_clean']
y = df['label']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
X_train.shape, X_test.shape


In [ ]:
tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1,2),
    stop_words='english'
)


In [ ]:
X_train_tfidf = tfidf.fit_transform(X_train)


In [ ]:
X_test_tfidf = tfidf.transform(X_test)


In [ ]:
X_train_tfidf.shape


In [ ]:
lr_model = LogisticRegression(max_iter=1000)


In [ ]:
lr_model.fit(X_train_tfidf, y_train)


In [ ]:
lr_preds = lr_model.predict(X_test_tfidf)


In [ ]:
accuracy_score(y_test, lr_preds)


In [ ]:
accuracy_score(y_test, lr_preds)


In [ ]:
print(classification_report(y_test, lr_preds))


In [ ]:
confusion_matrix(y_test, lr_preds)


In [ ]:
nb_model = MultinomialNB()


In [ ]:
nb_model.fit(X_train_tfidf, y_train)


In [ ]:
nb_preds = nb_model.predict(X_test_tfidf)


In [ ]:
accuracy_score(y_test, nb_preds)


In [ ]:
print(classification_report(y_test, nb_preds))


## Model Comparison
- Logistic Regression: Higher precision and recall
- Naive Bayes: Faster but slightly lower accuracy


In [ ]:
feature_names = tfidf.get_feature_names_out()
coefficients = lr_model.coef_[0]

top_positive = pd.Series(coefficients, index=feature_names).sort_values(ascending=False).head(20)
top_negative = pd.Series(coefficients, index=feature_names).sort_values().head(20)

top_positive, top_negative


In [ ]:
sample_tweet = ["you are an idiot"]
sample_vec = tfidf.transform(sample_tweet)
lr_model.predict(sample_vec)


In [ ]:
lr_model.predict_proba(sample_vec)


In [ ]:
import joblib

joblib.dump(lr_model, 'toxic_lr_model.pkl')
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')


In [ ]:
loaded_model = joblib.load('toxic_lr_model.pkl')
loaded_vectorizer = joblib.load('tfidf_vectorizer.pkl')


In [ ]:
loaded_model.predict(loaded_vectorizer.transform(sample_tweet))


## Final Insights
- TF-IDF + Logistic Regression performs best
- Text preprocessing significantly improves accuracy
- Model is suitable for real-time moderation systems


# ✅ Project Completed
This notebook now covers:
- Pandas EDA
- Text preprocessing
- Feature engineering
- ML modeling
- Evaluation
- Model saving


**Project:** Toxic Tweet Classification using Pandas & Machine Learning  
**Tech:** Pandas, NumPy, Scikit-learn, TF-IDF, Logistic Regression


# Thank You
